In [28]:
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input
import keras as k
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import SGD, RMSprop, Adam
from keras.layers.normalization import BatchNormalization

import numpy as np
import pandas as pd
import cv2
import shutil
from matplotlib import pyplot as plt
from tqdm import tqdm

In [33]:
DATA_DIR = '/home/chicm/ml/kgdata/species'
RESULT_DIR = DATA_DIR + '/results'

TRAIN_FEAT = RESULT_DIR + '/train_feats.dat'
VAL_FEAT = RESULT_DIR + '/val_feats.dat'

TRAIN_DIR = DATA_DIR + '/train-224'
VAL_DIR = DATA_DIR + '/val-224'

batch_size = 64

In [4]:
df_train = pd.read_csv(DATA_DIR+'/train_labels.csv')

## create validation data

In [6]:
f_dict = {row[0]: row[1] for i, row in enumerate(df_train.values)}
fnames = [row[0] for i, row in enumerate(df_train.values)]
print(len(f_dict))
print(fnames[:10])


2295
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


In [9]:
print(len([row[1] for i, row in enumerate(df_train.values) if row[1] == 0]))
print(len([row[1] for i, row in enumerate(df_train.values) if row[1] == 1]))

847
1448


In [14]:
for f in fnames:
    cls = f_dict[f]
    src = TRAIN_DIR + '/' + str(f) + '.jpg'
    dst = TRAIN_DIR + '/' + str(cls) + '/' + str(f) + '.jpg'
    shutil.move(src, dst)

In [17]:
fnames = np.random.permutation(fnames)
for i in range(350):
    cls = f_dict[fnames[i]]
    fn     = TRAIN_DIR +'/' + str(cls) + '/' + str(fnames[i])+'.jpg'
    tgt_fn = VAL_DIR   +'/' + str(cls) + '/' + str(fnames[i])+'.jpg'
    shutil.move(fn, tgt_fn)

## build pretrained model

In [25]:
vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

In [29]:
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=(7,7,512)))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

#model.add(top_model)
model = Model(inputs=vgg_model.input, outputs=top_model(vgg_model.output))

In [30]:
for layer in model.layers[:25]:
    layer.trainable = False

In [32]:
model.compile(Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

In [35]:
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [36]:
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary')

Found 1945 images belonging to 2 classes.


In [40]:
print(train_generator.n)
print(train_generator.samples)

1945
1945


In [37]:
validation_generator = test_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='binary')

Found 350 images belonging to 2 classes.


In [43]:
epochs = 50
model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.n//batch_size,
    verbose=2)

Epoch 1/50
12s - loss: 0.8021 - acc: 0.5160 - val_loss: 0.6967 - val_acc: 0.5385
Epoch 2/50
11s - loss: 0.8208 - acc: 0.4923 - val_loss: 0.6955 - val_acc: 0.5219
Epoch 3/50
11s - loss: 0.8057 - acc: 0.5085 - val_loss: 0.7009 - val_acc: 0.4755
Epoch 4/50
11s - loss: 0.8037 - acc: 0.4996 - val_loss: 0.6970 - val_acc: 0.5315
Epoch 5/50
11s - loss: 0.8136 - acc: 0.4905 - val_loss: 0.6909 - val_acc: 0.5315
Epoch 6/50
11s - loss: 0.8203 - acc: 0.4801 - val_loss: 0.6995 - val_acc: 0.5105
Epoch 7/50
11s - loss: 0.8204 - acc: 0.4881 - val_loss: 0.6966 - val_acc: 0.5156
Epoch 8/50


KeyboardInterrupt: 

In [52]:
def create_model():
    conv_layers = [
        Conv2D(24,(3,3), activation='relu',input_shape=(224,224,3)),
        BatchNormalization(axis=-1),
        MaxPooling2D((2, 2), strides=(2, 2)),

        Conv2D(24,(3,3), activation='relu'),
        BatchNormalization(axis=-1),
        MaxPooling2D((2, 2), strides=(2, 2)),

        Conv2D(48,(3,3), activation='relu'),
        BatchNormalization(axis=-1),
        MaxPooling2D((2, 2), strides=(2, 2)),

        Conv2D(48,(3,3), activation='relu'),
        BatchNormalization(axis=-1),
        MaxPooling2D((2, 2), strides=(2, 2)),
        
        Flatten(),
        
        Dropout(0.25),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(1, activation='softmax')
    ]
    #print conv_layers
    model = Sequential(conv_layers)
    model.compile(Adam(), loss = 'binary_crossentropy', metrics=['accuracy'])
    
    return model

In [53]:
epochs = 50
model2 = create_model()
model2.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=validation_generator.n//batch_size,
    verbose=2)

Epoch 1/50
13s - loss: 5.9748 - acc: 0.6252 - val_loss: 6.0759 - val_acc: 0.6189
Epoch 2/50
11s - loss: 5.7836 - acc: 0.6372 - val_loss: 6.4661 - val_acc: 0.5944
Epoch 3/50
11s - loss: 5.8003 - acc: 0.6362 - val_loss: 6.2989 - val_acc: 0.6049
Epoch 4/50
11s - loss: 5.7959 - acc: 0.6364 - val_loss: 5.8530 - val_acc: 0.6329
Epoch 5/50
12s - loss: 5.8289 - acc: 0.6344 - val_loss: 6.4104 - val_acc: 0.5979
Epoch 6/50
11s - loss: 5.8293 - acc: 0.6344 - val_loss: 5.6857 - val_acc: 0.6434
Epoch 7/50
12s - loss: 5.8378 - acc: 0.6338 - val_loss: 6.0202 - val_acc: 0.6224
Epoch 8/50


KeyboardInterrupt: 

In [54]:
x_train = []
y_train = []
for i, row in tqdm(enumerate(df_train.values)):
    fn = DATA_DIR+'/train/' + str(row[0])+'.jpg'
    x_train.append(cv2.resize(cv2.imread(fn), (224,224)))
    y_train.append([row[1]])
x_train = np.array(x_train, np.float32)
y_train = np.array(y_train, np.uint8)
print(x_train.shape)
print(y_train.shape)

2295it [00:46, 49.14it/s]


(2295, 224, 224, 3)
(2295, 1)


In [55]:
split = int(x_train.shape[0] * 0.85)

x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
print(x_train.shape)
print(x_val.shape)
print(y_train.shape)
print(y_val.shape)

(1950, 224, 224, 3)
(345, 224, 224, 3)
(1950, 1)
(345, 1)


In [28]:
train_steps = x_train.shape[0] // batch_size
x_train = x_train[:train_steps*batch_size]
y_train = y_train[:train_steps*batch_size]
val_steps = x_val.shape[0] // batch_size
x_val = x_val[:val_steps*batch_size]
y_val = y_val[:val_steps*batch_size]
print(train_steps)
print(val_steps)

30
5


In [56]:
model = VGG16(weights='imagenet', include_top=False)

In [30]:
datagen = ImageDataGenerator(
        rotation_range=45,
        width_shift_range=0.1,
        height_shift_range=0.1,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip = True)

In [44]:
print(x_train.shape)
print(x_val.shape)

(1920, 224, 224, 3)
(320, 224, 224, 3)


In [32]:
train_feat = model.predict_generator(datagen.flow(x_train, batch_size=batch_size, shuffle=False), 
                                     steps = train_steps*8) 

In [57]:
#val_feat = model.predict_generator(datagen.flow(x_val, batch_size=batch_size, shuffle=False), 
 #steps = val_steps) 
train_feat = model.predict(x_train)
val_feat = model.predict(x_val)

In [58]:
print(train_feat.shape)
print(val_feat.shape)

(1950, 7, 7, 512)
(345, 7, 7, 512)


In [59]:
import bcolz

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]

In [60]:
save_array(TRAIN_FEAT, train_feat)
save_array(VAL_FEAT, val_feat)

In [61]:
print(train_feat.shape)
print(val_feat.shape)

(1950, 7, 7, 512)
(345, 7, 7, 512)


In [62]:
def get_layers(input_shape):
    return [
        Flatten(input_shape=input_shape),
        Dropout(0.4),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.6),
        Dense(1, activation='sigmoid')
    ]

def get_model(input_shape):
    model = Sequential(get_layers(input_shape))
    model.compile(Adam(), loss = 'binary_crossentropy', metrics=['accuracy'])
    return model

In [63]:
dense_model = get_model(train_feat.shape[1:])

In [64]:
dense_model.fit(train_feat, y_train, batch_size=batch_size, 
                validation_data=(val_feat, y_val),
                epochs=50, verbose=2)

Train on 1950 samples, validate on 345 samples
Epoch 1/50
1s - loss: 0.5180 - acc: 0.7892 - val_loss: 0.3515 - val_acc: 0.8986
Epoch 2/50
0s - loss: 0.2507 - acc: 0.9000 - val_loss: 0.2008 - val_acc: 0.9391
Epoch 3/50
0s - loss: 0.1702 - acc: 0.9344 - val_loss: 0.2712 - val_acc: 0.9246
Epoch 4/50
0s - loss: 0.1175 - acc: 0.9559 - val_loss: 0.1826 - val_acc: 0.9333
Epoch 5/50
0s - loss: 0.0794 - acc: 0.9697 - val_loss: 0.2345 - val_acc: 0.9217
Epoch 6/50
0s - loss: 0.0673 - acc: 0.9769 - val_loss: 0.2085 - val_acc: 0.9246
Epoch 7/50
0s - loss: 0.0634 - acc: 0.9764 - val_loss: 0.2127 - val_acc: 0.9217
Epoch 8/50
0s - loss: 0.0434 - acc: 0.9851 - val_loss: 0.2152 - val_acc: 0.9304
Epoch 9/50
0s - loss: 0.0558 - acc: 0.9831 - val_loss: 0.1870 - val_acc: 0.9449
Epoch 10/50
0s - loss: 0.0464 - acc: 0.9800 - val_loss: 0.1804 - val_acc: 0.9333
Epoch 11/50
0s - loss: 0.0407 - acc: 0.9846 - val_loss: 0.2482 - val_acc: 0.9333
Epoch 12/50
0s - loss: 0.0375 - acc: 0.9856 - val_loss: 0.2167 - val_ac

In [52]:
y_train_da = np.concatenate([y_train]*8)
print(y_train.shape)
print(y_train_da.shape)

(1920, 1)
(15360, 1)


In [53]:
model = get_model(train_feat.shape[1:])

model.fit(train_feat, y_train_da, batch_size=batch_size, validation_data=(val_feat, y_val), epochs = 20, verbose=2)

Train on 15360 samples, validate on 320 samples
Epoch 1/20
3s - loss: 0.4335 - acc: 0.8275 - val_loss: 2.0344 - val_acc: 0.8250
Epoch 2/20
2s - loss: 0.2770 - acc: 0.8854 - val_loss: 2.9618 - val_acc: 0.7594
Epoch 3/20
2s - loss: 0.2355 - acc: 0.9029 - val_loss: 3.6947 - val_acc: 0.7375
Epoch 4/20
2s - loss: 0.2196 - acc: 0.9088 - val_loss: 3.4920 - val_acc: 0.7438
Epoch 5/20
2s - loss: 0.2015 - acc: 0.9156 - val_loss: 4.7506 - val_acc: 0.6937
Epoch 6/20
2s - loss: 0.1968 - acc: 0.9182 - val_loss: 4.1566 - val_acc: 0.7188
Epoch 7/20
2s - loss: 0.1873 - acc: 0.9236 - val_loss: 4.6893 - val_acc: 0.6906
Epoch 8/20
2s - loss: 0.1770 - acc: 0.9266 - val_loss: 4.7387 - val_acc: 0.6906
Epoch 9/20
2s - loss: 0.1673 - acc: 0.9315 - val_loss: 4.6118 - val_acc: 0.6937
Epoch 10/20
2s - loss: 0.1646 - acc: 0.9318 - val_loss: 4.6731 - val_acc: 0.7000
Epoch 11/20
2s - loss: 0.1546 - acc: 0.9363 - val_loss: 4.5431 - val_acc: 0.7000
Epoch 12/20
2s - loss: 0.1469 - acc: 0.9419 - val_loss: 4.5946 - val_a

In [41]:
model.optimizer.lr = 0.00001
model.fit(train_feat, y_train_da, batch_size=batch_size, validation_data=(val_feat, y_val), epochs = 50, verbose=2)

Train on 15360 samples, validate on 320 samples
Epoch 1/50
3s - loss: 0.1186 - acc: 0.9530 - val_loss: 0.3093 - val_acc: 0.8750
Epoch 2/50
2s - loss: 0.1270 - acc: 0.9470 - val_loss: 0.3046 - val_acc: 0.8906
Epoch 3/50
2s - loss: 0.1189 - acc: 0.9536 - val_loss: 0.2750 - val_acc: 0.8906
Epoch 4/50
2s - loss: 0.1239 - acc: 0.9516 - val_loss: 0.2809 - val_acc: 0.9000
Epoch 5/50
2s - loss: 0.1140 - acc: 0.9540 - val_loss: 0.3038 - val_acc: 0.8875
Epoch 6/50
2s - loss: 0.1139 - acc: 0.9544 - val_loss: 0.2951 - val_acc: 0.8812
Epoch 7/50
2s - loss: 0.1110 - acc: 0.9551 - val_loss: 0.3364 - val_acc: 0.8594
Epoch 8/50
2s - loss: 0.1141 - acc: 0.9548 - val_loss: 0.2855 - val_acc: 0.8812
Epoch 9/50
2s - loss: 0.1152 - acc: 0.9553 - val_loss: 0.3078 - val_acc: 0.9000
Epoch 10/50
2s - loss: 0.1052 - acc: 0.9585 - val_loss: 0.3077 - val_acc: 0.8844
Epoch 11/50
2s - loss: 0.1006 - acc: 0.9600 - val_loss: 0.3366 - val_acc: 0.8781
Epoch 12/50
2s - loss: 0.1073 - acc: 0.9568 - val_loss: 0.3699 - val_a